## Correlation

Correlation Feature Selection evaluates subsets of features on the basis of the following hypothesis: "Good feature subsets contain features highly correlated with the target, yet uncorrelated to each other".

**References**:

M. Hall 1999, [Correlation-based Feature Selection for Machine Learning](http://www.cs.waikato.ac.nz/~mhall/thesis.pdf)

Senliol, Baris, et al. "Fast Correlation Based Filter (FCBF) with a different search strategy." Computer and Information Sciences.



I will demonstrate how to select features based on correlation using 2 procedures. The first one is a brute force function that finds correlated features without any further insight. The second procedure finds groups of correlated features. Often, more than 2 features are correlated with each other. We can find groups of 3, 4 or more features that are correlated. By identifying these groups, we can then select from each group, which feature we want to keep, and which ones we want to remove.

I will use the Paribas claims dataset from Kaggle.

In [ ]:
!pip install --user kaggle

In [ ]:
!mkdir .kaggle


In [ ]:
import json
token = {"username":"######","key":"############"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c bnp-paribas-cardif-claims-management

 67% 33.0M/49.4M [00:01<00:01, 13.7MB/s]
100% 49.4M/49.4M [00:01<00:00, 42.5MB/s]
  0% 0.00/162k [00:00<?, ?B/s]
100% 162k/162k [00:00<00:00, 143MB/s]
 69% 34.0M/49.4M [00:00<00:00, 162MB/s]
100% 49.4M/49.4M [00:00<00:00, 165MB/s]


In [ ]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [ ]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

In [ ]:
# load dataset
data = pd.read_csv('train.csv', nrows=50000)
data.shape

(50000, 133)

In [ ]:
data.head()

ID  target        v1        v2  ...      v128  v129      v130      v131
0   3       1  1.335739  8.727474  ...  2.024285     0  0.636365  2.857144
1   4       1       NaN       NaN  ...  1.957825     0       NaN       NaN
2   5       1  0.943877  5.310079  ...  1.120468     2  0.883118  1.176472
3   6       1  0.797415  8.304757  ...  1.990847     1  1.677108  1.034483
4   8       1       NaN       NaN  ...       NaN     0       NaN       NaN

[5 rows x 133 columns]

In [ ]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess whether they are correlated with other features

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(50000, 114)

### Important

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [ ]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target', 'ID'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 112), (15000, 112))

In [ ]:
# visualise correlated features
# I will build the correlation matrix, which examines the 
# correlation of all features (for all possible feature combinations)
# and then visualise the correlation matrix using seaborn

corrmat = X_train.corr()
fig, ax = plt.subplots()
fig.set_size_inches(11,11)
sns.heatmap(corrmat)

In the plot above, the red squares correspond to highly correlated features (>0.8). We can see that there are quite a few. The diagonal represents the correlation of a feature with itself, therefore the value is 1.

### Brute force approach

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything else
# without any other insight.

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train, 0.8)
len(set(corr_features))

55

We can see that 55 features are highly correlated with other features in the training set. Very likely, by removing these correlated features, the performance of your machine learning models will drop very little, if at all. We can go ahead and drop the features like we have done in previous lectures.

In [ ]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 57), (15000, 57))

By removing correlated columns we reduced the feature space from 112 numerical columns to 57, almost half of the original set.

### Second approach

The second approach looks to identify groups of highly correlated features. And then, we can make further investigation within these groups to decide which feature we keep and which one we remove.

In [ ]:
# load dataset
data = pd.read_csv('train.csv', nrows=50000)

# select numerical variables
data = data[numerical_vars]

# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target', 'ID'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

In [ ]:
# build a dataframe with the correlation between features
# remember that the absolute value of the correlation
# coefficient is important and not the sign

corrmat = X_train.corr()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[corrmat >= 0.8]
corrmat = corrmat[corrmat < 1]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat.head()

feature1 feature2      corr
0     v100      v58  0.997974
1      v58     v100  0.997974
2     v115      v69  0.993229
3      v69     v115  0.993229
4      v17      v76  0.992511

In [ ]:
# find groups of correlated features

grouped_feature_ls = []
correlated_groups = []

for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:

        # find all features correlated to a single feature
        correlated_block = corrmat[corrmat.feature1 == feature]
        grouped_feature_ls = grouped_feature_ls + list(
            correlated_block.feature2.unique()) + [feature]

        # append the block of features to the list
        correlated_groups.append(correlated_block)

print('found {} correlated groups'.format(len(correlated_groups)))
print('out of {} total features'.format(X_train.shape[1]))

found 32 correlated groups
out of 112 total features


In [ ]:
# now we can visualise each group. We see that some groups contain
# only 2 correlated features, some other groups present several features 
# that are correlated among themselves.

for group in correlated_groups:
    print(group)
    print()

  feature1 feature2      corr
0     v100      v58  0.997974

  feature1 feature2      corr
2     v115      v69  0.993229

    feature1 feature2      corr
4        v17      v76  0.992511
9        v17      v64  0.982145
107      v17      v48  0.910103
130      v17      v93  0.887841
174      v17     v106  0.854995
218      v17      v44  0.818050
243      v17     v101  0.800001

   feature1 feature2      corr
6       v46      v63  0.991521
10      v46      v25  0.982115
24      v46       v8  0.974733
58      v46     v105  0.944465
72      v46      v89  0.932504
88      v46      v54  0.918355

    feature1 feature2      corr
16       v60      v26  0.980037
148      v60      v43  0.872723
176      v60     v116  0.854290

   feature1 feature2      corr
18      v92      v95  0.978357

   feature1 feature2      corr
28     v114      v40  0.966750
99     v114      v34  0.911422

    feature1 feature2      corr
30       v83     v121  0.966052
35       v83      v33  0.964033
115      v83     v111

In [ ]:
# we can now investigate further features within one group.
# let's for example select group 3

group = correlated_groups[2]
group

feature1 feature2      corr
4        v17      v76  0.992511
9        v17      v64  0.982145
107      v17      v48  0.910103
130      v17      v93  0.887841
174      v17     v106  0.854995
218      v17      v44  0.818050
243      v17     v101  0.800001

In [ ]:
# we could select the features with less missing data
# like this:

for feature in list(group.feature2.unique())+['v17']:
    print(X_train[feature].isnull().sum())

15275
15275
15275
15288
15275
15275
15275
15275


All features show similar number of missing data in this case.

Alternatively, we could build a machine learning algorithm using all the features from the above list, and select the more predictive one.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = list(group.feature2.unique())+['v17']
rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
rf.fit(X_train[features].fillna(0), y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=39, verbose=0,
                       warm_start=False)

In [ ]:
# we get the feature importance attributed by the 
# random forest model (more on this in coming lectures)

importance = pd.concat(
    [pd.Series(features),
     pd.Series(rf.feature_importances_)], axis=1)

importance.columns = ['feature', 'importance']
importance.sort_values(by='importance', ascending=False)

feature  importance
2     v48    0.173981
3     v93    0.154484
6    v101    0.129764
1     v64    0.118110
7     v17    0.117571
4    v106    0.113958
0     v76    0.108071
5     v44    0.084062

In this case, feature v48 shows the higher importance according to random forests. Then, I would select v48, and remove all the remaining features from this group from the dataset.

**Note**

None of the 2 procedures for removing correlated features are perfect, and some correlated features may escape the loops of code. So it might be worthwhile to check that after removing the correlated features, there are no correlated features left in the dataset. If there are, repeat the procedure to remove the remaining ones.

